In [1]:
import os

os.chdir("..")


In [5]:

from ipycanvas import Canvas
from PIL import Image
import numpy as np
import time

import cv2

image_path = 'assets/scene_01.png'
# Use an example image path or replace with your own
save_path = 'current_canvas.png'


is_mouse_down = False
position_at_mouse_down = None
position_at_mouse_up = None
image = None
canvas = Canvas(width=256, height=256, sync_image_data=True)


def draw_rectangle(x0, y0, x1, y1, color='red', fill=False):
    canvas.stroke_style = color
    canvas.line_width = 3
    canvas.stroke_rect(min(x0, x1), min(y0, y1), abs(x1 - x0), abs(y1 - y0))
    if fill:
        canvas.fill_style = color
        canvas.fill_rect(min(x0, x1), min(y0, y1), abs(x1 - x0), abs(y1 - y0))
        
# Draw with red brush when dragging
def handle_mouse_down(x, y):
    global is_mouse_down, position_at_mouse_down
    is_mouse_down = True
    position_at_mouse_down = (x, y)
    
def handle_mouse_up(x, y):
    global is_mouse_down, position_at_mouse_up, image
    is_mouse_down = False
    position_at_mouse_up = (x, y)
    if position_at_mouse_down is not None:
        # Draw the rectangle directly on the image array and update the canvas
        img_arr = np.array(image)
        x0, y0 = position_at_mouse_down
        x1, y1 = x, y
        # Draw rectangle on the numpy array (red color)
        cv2.rectangle(
            img_arr,
            (int(x0), int(y0)),
            (int(x1), int(y1)),
            color=(255, 255, 255, 127),  # Red in RGB
            thickness=-1
        )
        image = Image.fromarray(img_arr)
        canvas.put_image_data(np.array(image))
        # image = Image.fromarray(canvas.get_image_data())
        image.save(save_path)



def handle_mouse_move(x, y):
    if not is_mouse_down:
        return
    # Draw rectangle from position_at_mouse_down to current (x, y)
    if position_at_mouse_down is not None:
        # Clear and redraw the image
        canvas.put_image_data(np.array(image))
        draw_rectangle(position_at_mouse_down[0], position_at_mouse_down[1], x, y)
        pass

# canvas.observe(save_to_file, "image_data")
canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_move(handle_mouse_move)
canvas.on_mouse_up(handle_mouse_up)


def edit_image(image_path, _save_path = 'current_canvas.png'):
    global image, save_path
    save_path = _save_path
    image = Image.open(image_path).convert("RGB")
    orig_w, orig_h = image.size
    max_side = 512
    if orig_w >= orig_h:
        new_w = max_side
        new_h = int(orig_h * max_side / orig_w)
    else:
        new_h = max_side
        new_w = int(orig_w * max_side / orig_h)
    image = image.resize((new_w, new_h), Image.LANCZOS)
    canvas.width = new_w
    canvas.height = new_h
    # Draw image on canvas
    canvas.put_image_data(np.array(image))

    return canvas

In [7]:

image_paths = ['assets/scene_01.png', 'assets/scene_02.png']  # Replace with your image path

edit_image(image_paths[1], "abc2.png")


Canvas(height=276, image_data=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x00\x00\x00\x01\x14\x08\x06\x0…